In [1]:
import os
import config as c
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.datasets import fetch_california_housing
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go

from visualization.utils import plot_correlation_matrix, plot_scatterplot_matrix
from utils.metrics import NIC

init_notebook_mode(connected=True)

warnings.filterwarnings('ignore')
plt.rcParams ['figure.figsize'] = (13,8)
sns.set()
%config InlineBackend.figure_format = 'retina'

In [275]:
pd.set_option('display.max_row', 20)

In [72]:
df_trxnl

,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2
2378974,-3350320981274421332,5178654056819692781,2018-09-01 00:00:00,1290.0,5541.0,624547912,RUS,LANGEPAS,AZS EKOTEK,Payment for goods and services,NaN
2361041,5472108446909007331,2413753720180261573,2018-09-01 00:00:00,248.0,5499.0,624547912,RUS,SARATOV,YA,Payment for goods and services,NaN
2505978,6512635731274681123,-8080160968658314781,2018-09-01 00:00:00,1821.0,5651.0,624547912,RUS,NOVOSIBIRSK,IP KOROTKIKH YA.A.,Payment for goods and services,NaN
2394118,-2418310972695527328,-5697703739274576066,2018-09-01 00:00:00,571.0,5499.0,624547912,RUS,KHANTYMANSIJ,IP GACHIMOV,Payment for goods and services,NaN
2212465,6898413714332962934,1707902273228688352,2018-09-01 00:00:00,9106.0,6011.0,624547912,RUS,ROSTOVNADON,A DNEPROVSKIY BYSTR,Cash withdrawal through an ATM,NaN
...,...,...,...,...,...,...,...,...,...,...,...
66437,-6187341189294406525,-595439369881602459,2019-08-31 23:55:07,19.0,5462.0,-146132545,TUR,ANTALYA,AKSOY SIMIT,Payment for goods and services,NaN
371140,1452416481678292939,8941200550571135837,2019-08-31 23:55:16,216.0,5691.0,624547912,RUS,MILKOVO,WB,Payment for goods and services,NaN
354072,5655208186499508244,2271445589626924309,2019-08-31 23:56:30,607.0,5947.0,1741018794,RUS,MOSKVA,WALLETONE,Payment for goods and services,NaN
426114,8210223685678480315,8904613667186575982,2019-08-31 23:57:12,73.0,7995.0,624547912,RUS,MOSCOW,YM STOLOTO,Payment for goods and services,NaN


In [220]:
df_trxnl = pd.read_csv(os.path.join(c.DATASET_DIR, 'trxn.csv'))
df_trxnl.tran_time = pd.to_datetime(df_trxnl.tran_time)
df_trxnl = df_trxnl.sort_values(by=['tran_time'])

df_trxnl.mcc_cd = df_trxnl.mcc_cd.astype('category')
df_trxnl.merchant_cd = df_trxnl.merchant_cd.astype('int').astype('category')
df_trxnl.head()

,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2
2378974,-3350320981274421332,5178654056819692781,2018-09-01,1290.0,5541.0,624547912,RUS,LANGEPAS,AZS EKOTEK,Payment for goods and services,NaN
2361041,5472108446909007331,2413753720180261573,2018-09-01,248.0,5499.0,624547912,RUS,SARATOV,YA,Payment for goods and services,NaN
2505978,6512635731274681123,-8080160968658314781,2018-09-01,1821.0,5651.0,624547912,RUS,NOVOSIBIRSK,IP KOROTKIKH YA.A.,Payment for goods and services,NaN
2394118,-2418310972695527328,-5697703739274576066,2018-09-01,571.0,5499.0,624547912,RUS,KHANTYMANSIJ,IP GACHIMOV,Payment for goods and services,NaN
2212465,6898413714332962934,1707902273228688352,2018-09-01,9106.0,6011.0,624547912,RUS,ROSTOVNADON,A DNEPROVSKIY BYSTR,Cash withdrawal through an ATM,NaN


In [44]:
df_appl = pd.read_csv(os.path.join(c.DATASET_DIR, 'appl.csv'))
df_appl.month_end_dt = pd.to_datetime(df_appl.month_end_dt)
df_appl.head()

,client_id,month_end_dt,appl_prod_group_name,appl_prod_type_name,appl_stts_name_dc,appl_sale_channel_name
0,6886062013213911831,2018-10-31,PILS,PILS,The consultation is over,Network
1,4497537671718296975,2018-10-31,PILS,PILS,The consultation is over,Network
2,1316154201527307743,2018-09-30,PILS,PILS,The consultation is over,Network
3,1316154201527307743,2018-09-30,PILS,PILS,The consultation is over,Network
4,1316154201527307743,2018-09-30,PILS,PILS,The consultation is over,Network


In [190]:
df_balance = pd.read_csv(os.path.join(c.DATASET_DIR, 'balance.csv'))
df_balance.month_end_dt = pd.to_datetime(df_balance.month_end_dt)
df_balance = df_balance.sort_values(by=['month_end_dt'])
df_balance.head()

,client_id,crncy_cd,eop_bal_sum_rur,min_bal_sum_rur,max_bal_sum_rur,avg_bal_sum_rur,month_end_dt,prod_cat_name,prod_group_name
0,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
392810,6562924311126763210,810.0,0.0,0.0,0.0,0.0,2018-09-30,DEPOSITS,Time deposits
932309,7780105290539679707,810.0,0.0,0.0,0.0,0.0,2018-09-30,KN,NaN
392844,-3752205643827810202,810.0,0.0,0.0,0.0,0.0,2018-09-30,DEPOSITS,Time deposits
392845,-3752205643827810202,810.0,0.0,0.0,0.0,0.0,2018-09-30,DEPOSITS,Time deposits


In [63]:
df_com = pd.read_csv(os.path.join(c.DATASET_DIR, 'com.csv'))
df_com.month_end_dt = pd.to_datetime(df_com.month_end_dt)
df_com.head()

,client_id,agr_flg,otkaz,dumaet,ring_up_flg,not_ring_up_flg,count_comm,month_end_dt,channel,prod
0,7513301859607023584,0,0,0,0,2,2,2018-12-31,CALL,Credit Card
1,7513301859607023584,0,0,0,0,2,2,2019-02-28,CALL,Credit Card
2,-8079817404148237243,0,0,0,2,0,2,2019-08-31,CALL,Credit Card
3,-8079817404148237243,0,0,0,2,0,2,2019-03-31,CALL,Credit Card
4,2897849887287086776,0,0,0,0,16,16,2019-02-28,CALL,Credit Card


In [65]:
df_client = pd.read_csv(os.path.join(c.DATASET_DIR, 'client.csv'))
df_client.head()

,client_id,gender,age,region,city,citizenship,education,job_type
0,7513301859607023584,F,33.0,0,115,RUSSIA,NaN,NaN
1,9157009756404187626,F,59.0,17,668,RUSSIA,NaN,NaN
2,-1893104556496814867,M,51.0,28,65,RUSSIA,NaN,NaN
3,6886062013213911831,F,56.0,0,40,RUSSIA,NaN,NaN
4,-8156468515495593794,F,34.0,-1,-1,RUSSIA,HIGHER_PROFESSIONAL,NaN


In [86]:
df_deals = pd.read_csv(os.path.join(c.DATASET_DIR, 'deals.csv'))
df_deals.agrmnt_start_dt = pd.to_datetime(df_deals.agrmnt_start_dt)
df_deals = df_deals.sort_values(by=['agrmnt_start_dt'])
df_deals.head()

,client_id,agrmnt_start_dt,agrmnt_close_dt,crncy_cd,agrmnt_rate_active,agrmnt_rate_passive,agrmnt_sum_rur,prod_type_name
5203,4747392352022104823,1990-05-14,NaN,810.0,NaN,0.01,0.0,Cash on demand
4154,5728212225549374245,1990-07-16,2015-10-26,810.0,NaN,0.10,1302.0,Cash on demand
2212,-4619525372465538922,1990-08-16,2015-10-27,810.0,NaN,0.10,6.0,Cash on demand
8714,6359952416951480141,1990-12-09,NaN,810.0,NaN,0.01,0.0,Cash on demand
8943,-2139121420305146462,1991-05-19,2015-11-02,810.0,NaN,0.10,34.0,Cash on demand


In [87]:
df_payments = pd.read_csv(os.path.join(c.DATASET_DIR, 'payments.csv'))
df_payments.day_dt = pd.to_datetime(df_payments.day_dt)
df_payments = df_payments.sort_values(by=['day_dt'])
df_payments.head()

,client_id,day_dt,sum_rur,pmnts_name
134255,-3455628785094032071,2018-09-02,12141.0,Salary receipts
49583,1856365456049300130,2018-09-02,6848.0,Salary receipts
120137,-5326803468345248682,2018-09-02,18951.0,Salary receipts
8086,-6042851447286911222,2018-09-02,7378.0,Salary receipts
26053,2387040020794760226,2018-09-02,53605.0,Salary receipts


In [319]:
df_funnel = pd.read_csv(os.path.join(c.DATASET_DIR, 'funnel.csv'))
df_funnel = pd.concat([df_funnel.set_index('client_id'), df_client.set_index('client_id')], axis=1).reset_index()
df_funnel.head()

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,...,feature_9,feature_10,region_cd,gender,age,region,city,citizenship,education,job_type
0,7513301859607023584,0,NaN,1,7,13.0,571533.0,15717.0,0.0,0.0,...,10.4,12548.0,86.0,F,33.0,0,115,RUSSIA,NaN,NaN
1,9157009756404187626,0,NaN,1,3,13.0,3642369.0,94787.0,0.0,0.0,...,8.9,77210.0,2.0,F,59.0,17,668,RUSSIA,NaN,NaN
2,-1893104556496814867,0,NaN,1,5,16.0,352826.0,5500.0,0.0,6822.0,...,8.9,5508.0,52.0,M,51.0,28,65,RUSSIA,NaN,NaN
3,6886062013213911831,0,NaN,1,4,3.0,6070615.0,40580.0,0.0,30401.0,...,7.9,40583.0,86.0,F,56.0,0,40,RUSSIA,NaN,NaN
4,-8156468515495593794,1,138018.05,1,7,14.0,3642369.0,97156.0,81488.0,0.0,...,10.4,78108.0,27.0,F,34.0,-1,-1,RUSSIA,HIGHER_PROFESSIONAL,NaN


# Создание фич

In [332]:
df_trxnl

,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2
2378974,-3350320981274421332,5178654056819692781,2018-09-01 00:00:00,1290.0,5541.0,624547912,RUS,LANGEPAS,AZS EKOTEK,Payment for goods and services,NaN
2361041,5472108446909007331,2413753720180261573,2018-09-01 00:00:00,248.0,5499.0,624547912,RUS,SARATOV,YA,Payment for goods and services,NaN
2505978,6512635731274681123,-8080160968658314781,2018-09-01 00:00:00,1821.0,5651.0,624547912,RUS,NOVOSIBIRSK,IP KOROTKIKH YA.A.,Payment for goods and services,NaN
2394118,-2418310972695527328,-5697703739274576066,2018-09-01 00:00:00,571.0,5499.0,624547912,RUS,KHANTYMANSIJ,IP GACHIMOV,Payment for goods and services,NaN
2212465,6898413714332962934,1707902273228688352,2018-09-01 00:00:00,9106.0,6011.0,624547912,RUS,ROSTOVNADON,A DNEPROVSKIY BYSTR,Cash withdrawal through an ATM,NaN
...,...,...,...,...,...,...,...,...,...,...,...
66437,-6187341189294406525,-595439369881602459,2019-08-31 23:55:07,19.0,5462.0,-146132545,TUR,ANTALYA,AKSOY SIMIT,Payment for goods and services,NaN
371140,1452416481678292939,8941200550571135837,2019-08-31 23:55:16,216.0,5691.0,624547912,RUS,MILKOVO,WB,Payment for goods and services,NaN
354072,5655208186499508244,2271445589626924309,2019-08-31 23:56:30,607.0,5947.0,1741018794,RUS,MOSKVA,WALLETONE,Payment for goods and services,NaN
426114,8210223685678480315,8904613667186575982,2019-08-31 23:57:12,73.0,7995.0,624547912,RUS,MOSCOW,YM STOLOTO,Payment for goods and services,NaN


In [450]:
df_trxnl['year_month'] = df_trxnl.tran_time.dt.month + df_trxnl.tran_time.dt.year * 10

month_costs = df_trxnl.groupby(['client_id', 'year_month']).sum()['tran_amt_rur'].reset_index()
month_costs = month_costs.rename(columns={'tran_amt_rur': 'mean_month_costs'})
month_costs = month_costs.groupby('client_id').mean()['mean_month_costs'].reset_index()
df_funnel = pd.concat([df_funnel.set_index('client_id'), month_costs.set_index('client_id')], axis=1).reset_index()

In [455]:
month_payments = df_payments.groupby('client_id').mean()['sum_rur'].reset_index()
month_payments.rename(columns={'sum_rur': 'month_payments'})

df_funnel = pd.concat([df_funnel.set_index('client_id'), month_payments.set_index('client_id')], axis=1).reset_index()

In [401]:
df_funnel['delta'] = df_funnel.sum_rur - df_funnel.tran_amt_rur

In [402]:
df_funnel.corr()

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,...,feature_8,feature_9,feature_10,region_cd,age,region,city,tran_amt_rur,sum_rur,delta
client_id,1.000000,0.002857,0.024852,0.001679,-0.002944,-0.002263,0.006095,0.014837,0.002039,0.006921,...,0.008229,-0.002983,0.010589,-0.006946,-0.007511,-0.007042,0.000983,0.011113,0.012488,-0.018530
sale_flg,0.002857,1.000000,NaN,0.026489,-0.038500,-0.301903,0.061878,0.023095,0.080969,0.124936,...,0.102033,-0.169604,0.133996,0.031090,-0.023028,-0.056472,-0.098920,0.045029,0.018338,-0.046271
sale_amount,0.024852,NaN,1.000000,-0.014242,-0.048497,-0.054515,0.153004,0.355598,0.443969,0.180681,...,0.326322,-0.153427,0.421133,0.169763,0.094231,-0.146293,-0.104871,0.244278,0.214024,-0.230047
contacts,0.001679,0.026489,-0.014242,1.000000,0.000061,-0.127788,0.056231,0.009877,0.043545,0.023176,...,0.044323,-0.044366,0.053606,0.025433,-0.023065,-0.035442,-0.017541,-0.016523,-0.022700,0.012960
feature_1,-0.002944,-0.038500,-0.048497,0.000061,1.000000,-0.021809,-0.076170,0.007475,0.022636,0.040125,...,0.020056,0.364472,0.034297,0.000467,-0.023780,-0.003665,0.003439,0.029223,0.040027,-0.022731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
region,-0.007042,-0.056472,-0.146293,-0.035442,-0.003665,0.086111,-0.087676,-0.023066,-0.027156,-0.115405,...,-0.150888,0.093235,-0.150904,-0.379073,-0.006608,1.000000,0.216318,-0.044136,-0.098197,0.039379
city,0.000983,-0.098920,-0.104871,-0.017541,0.003439,0.040424,-0.057365,-0.041689,-0.041988,-0.057118,...,-0.109249,0.059237,-0.110128,-0.227713,-0.015504,0.216318,1.000000,-0.051339,-0.067467,0.051552
tran_amt_rur,0.011113,0.045029,0.244278,-0.016523,0.029223,-0.042389,0.059634,0.147651,0.123501,0.244396,...,0.210783,-0.012078,0.223511,0.039404,-0.039863,-0.044136,-0.051339,1.000000,0.245065,-0.982651
sum_rur,0.012488,0.018338,0.214024,-0.022700,0.040027,-0.067893,0.408856,0.776815,0.203548,0.889357,...,0.387621,-0.075298,0.394329,0.074781,-0.014582,-0.098197,-0.067467,0.245065,1.000000,-0.061003


In [446]:
features

Index(['feature_1', 'client_segment', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'region_cd', 'age', 'tran_amt_rur', 'sum_rur', 'delta'],
      dtype='object')

In [444]:
features = df_funnel.columns[4:]

In [445]:
features = features.drop(['region', 'gender', 'city', 'citizenship', 'education', 'job_type'])

In [437]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

model = CatBoostClassifier(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')
model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)
model.score(x_test, y_test)

0.8021474385530034

In [442]:
features = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'client_segment', 'region_cd']

In [443]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

model = CatBoostClassifier(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')
model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)
model.score(x_test, y_test)

0.8005380704638347

In [416]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)

score = 0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

    model = CatBoostRegressor(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')

    model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)

    score += NIC(model.predict(x), df_funnel.sale_amount, df_funnel.contacts, 0.55)
    
print('==||=='*10)
print(score / 10)

==||====||====||====||====||====||====||====||====||====||==
87.83901737000001


In [417]:
features = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'client_segment', 'region_cd']

In [418]:
x = df_funnel[features]
y = df_funnel.sale_flg.to_numpy().reshape(-1, 1)

score = 0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

    model = CatBoostRegressor(iterations=100,
                              learning_rate=3e-2,
                              l2_leaf_reg=3.0,  # any pos value
                              depth=5,  # int up to 16
                              min_data_in_leaf=1,  # 1,2,3,4,5
                              rsm=1,  # 0.01 .. 1.0
                              langevin=False,
                              task_type="GPU",
                              devices='0:1')

    model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)

    score += NIC(model.predict(x), df_funnel.sale_amount, df_funnel.contacts, 0.5)
    
print('==||=='*10)
print(score / 10)

==||====||====||====||====||====||====||====||====||====||==
87.89666834


In [462]:
kf = KFold(n_splits=10)
for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [403]:
features = df_funnel.columns[4:]

In [464]:
np.mean(df_funnel.contacts)

1.2265792166713183

In [302]:
from sklearn.metrics import mean_absolute_error

In [303]:
x = df_funnel[df_funnel.sale_amount.fillna(0) != 0][features]
y = df_funnel[df_funnel.sale_amount.fillna(0) != 0].sale_amount.to_numpy().reshape(-1, 1)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

model = CatBoostRegressor(iterations=100,
                          learning_rate=3e-2,
                          l2_leaf_reg=3.0,  # any pos value
                          depth=5,  # int up to 16
                          min_data_in_leaf=1,  # 1,2,3,4,5
                          rsm=1,  # 0.01 .. 1.0
                          langevin=False,
                          task_type="GPU",
                          devices='0:1')

model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose=0)

print(model.predict(x) - y)

[[  -2889.54100897   40174.89651451   -1725.83336481 ...   35184.75600508
    31302.83140168   36645.46463547]
 [ -35810.49100897    7253.94651451  -34646.78336481 ...    2263.80600508
    -1618.11859832    3724.51463547]
 [  21753.20899103   64817.64651451   22916.91663519 ...   59827.50600508
    55945.58140168   61288.21463547]
 ...
 [-126821.19100897  -83756.75348549 -125657.48336481 ...  -88746.89399492
   -92628.81859832  -87286.18536453]
 [  -9473.69100897   33590.74651451   -8309.98336481 ...   28600.60600508
    24718.68140168   30061.31463547]
 [  -9772.79100897   33291.64651451   -8609.08336481 ...   28301.50600508
    24419.58140168   29762.21463547]]


In [314]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(x.fillna(0), y)

In [ ]:
model.score(x_test, y_test)

In [317]:
mean_absolute_error(y, model.predict(x.fillna(0)))

24748.28564380754

In [318]:
mean_absolute_error(y, model.predict(x.fillna(0))*0+25000)

30057.40585393578